In [1]:
import os
import pandas as pd

# Change the working directory to strategy_lab
os.chdir("/Users/sugang/Desktop/projects/2024coin/strategy_lab")

# Now, you can read the CSV file without specifying the full path
df = pd.read_csv("temp.csv")
df

,Unnamed: 0,market,time_utc,open,high,low,close,volume_krw,volume_market
0,0,KRW-BTC,2017-09-25 03:00:00,4201000.0,4244000.0,4191000.0,4202000.0,4.142651e+08,98.210406
1,1,KRW-BTC,2017-09-25 04:00:00,4222000.0,4235000.0,4208000.0,4235000.0,3.230358e+07,7.656490
2,2,KRW-BTC,2017-09-25 08:00:00,4235000.0,4235000.0,4235000.0,4235000.0,1.270500e+02,0.000030
3,3,KRW-BTC,2017-09-25 10:00:00,4213000.0,4213000.0,4187000.0,4204000.0,3.832582e+06,0.914286
4,4,KRW-BTC,2017-09-25 11:00:00,4204000.0,4204000.0,4175000.0,4191000.0,9.211909e+06,2.197385
...,...,...,...,...,...,...,...,...,...
62439,62439,KRW-BTC,2024-11-13 04:00:00,124943000.0,125620000.0,122800000.0,124630000.0,8.196520e+10,661.005575
62440,62440,KRW-BTC,2024-11-13 05:00:00,124630000.0,124940000.0,122771000.0,122899000.0,5.244840e+10,423.694103
62441,62441,KRW-BTC,2024-11-13 06:00:00,122897000.0,124081000.0,122894000.0,123265000.0,7.074221e+10,573.349227
62442,62442,KRW-BTC,2024-11-13 07:00:00,123246000.0,124899000.0,123200000.0,124759000.0,5.841097e+10,470.626813


### make it daily data

In [2]:
from handle_candle import resample_df
from datetime import datetime

execution_time = datetime.strptime("00:00", "%H:%M")
df = resample_df(df, execution_time=execution_time)
df

,time_utc,open,high,low,close,volume_krw,volume_market
0,2017-09-25,4201000.0,4333000.0,4175000.0,4322000.0,5.602146e+08,132.484755
1,2017-09-26,4317000.0,4418000.0,4311000.0,4321000.0,9.950724e+07,22.788340
2,2017-09-27,4322000.0,4677000.0,4318000.0,4657000.0,1.448276e+08,32.269662
3,2017-09-28,4657000.0,4772000.0,4519000.0,4586000.0,3.721860e+08,80.588243
4,2017-09-29,4586000.0,4709000.0,4476000.0,4657000.0,2.724558e+08,59.352373
...,...,...,...,...,...,...,...
2602,2024-11-09,106845000.0,106990000.0,105873000.0,106294000.0,2.876788e+11,2702.452888
2603,2024-11-10,106294000.0,112470000.0,106050000.0,111440000.0,8.895481e+11,8111.036561
2604,2024-11-11,111440000.0,125073000.0,111273000.0,124503000.0,1.413171e+12,12056.831324
2605,2024-11-12,124476000.0,128010000.0,118864000.0,124967000.0,2.115376e+12,17037.759563


In [3]:
df["ma_5"] = df["close"].rolling(window=5).mean()
df["ma_20"] = df["close"].rolling(window=20).mean()

In [4]:
# Initialize the signal column with default value 0
df["signal"] = 0

# Generate buy signal: 1 when ma5 crosses above ma20
df.loc[(df["ma_5"] > df["ma_20"]) & (df["ma_5"].shift(1) <= df["ma_20"].shift(1)), "signal"] = 1

# Generate sell signal: -1 when ma_5 crosses below ma_20
df.loc[(df["ma_5"] < df["ma_20"]) & (df["ma_5"].shift(1) >= df["ma_20"].shift(1)), "signal"] = -1


# Initialize the position column with default value 0
df["position"] = 0

# Variable to track the current position status
current_position = 0

# Iterate over the rows and set the position
# How do I vectorize this..
for i in range(1, len(df)):
    if df.loc[i-1, "signal"] == 1:  # Enter long position
        if df.loc[i, "signal"] == -1:
            df.loc[i, "position"] = 1
            current_position =0
            continue
        current_position = 1
    elif df.loc[i, "signal"] == -1:  # Exit position
        df.loc[i, "position"] = current_position
        current_position = 0
        continue
    df.loc[i, "position"] = current_position



In [5]:
# Calculate the strategy returns (only when in a long position)
df["strategy_returns"] = df["position"] * df["close"].pct_change()
df["strategy_returns2"] = df["strategy_returns"]

for i in range(1, len(df)):
    buy_price = df.loc[i - 1, "close"]
    buy_price_copy = buy_price
    sell_price = df.loc[i, "close"]
    sell_price_copy = sell_price
    if df.loc[i - 1, "position"] == 0 and df.loc[i - 1, "signal"] == 1:
        # df.loc[i, 'strategy_returns2'] = (df.loc[i,'position'])/(df.loc[i-1, 'position'] * 1.002) -1
        buy_price = df.loc[i - 1, "close"] * 1.002
    if df.loc[i, "position"] == 1 and df.loc[i, "signal"] == -1:
        # df.loc[i, 'strategy_returns2'] = (df.loc[i,'position'] * 0.998)/(df.loc[i-1, 'position']) -1
        sell_price = df.loc[i, "close"] * 0.998

    if buy_price == buy_price_copy and sell_price == sell_price_copy:
        continue

    df.loc[i, "strategy_returns2"] = sell_price / buy_price - 1

# Calculate the cumulative returns
df["cumulative_returns"] = (1 + df["strategy_returns"]).cumprod()
df["cumulative_returns2"] = (1 + df["strategy_returns2"]).cumprod()

df.to_csv("1/temp3.csv")

In [6]:
from performance_utils import get_performance

print(get_performance(df))

{'total_return': 29.748233792090133, 'cagr': 0.6211960415567446, 'mdd': 0.6179259638751569, 'win_rate': 0.38235294117647056, 'gain_loss_ratio': 4.59, 'holding_time_ratio': 0.5347758887171561, 'investing_period': 2588}
